<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/langgraph-rag-agent-local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python sentence-transformers

In [2]:
import sys
print(sys.executable)

/home/aditya/miniconda3/bin/python


# Local LangGraph RAG agent with Llama 3

```

```


![Alternative text](./overview.jpeg)

We follow the following workflow in an attempt to make the query job more grounded in reality and stop the chatbot from straying away from relevance of the questoin

We use ollama to use local model of llama3


In [ ]:
! pip install langchain-nomic
! ollama pull llama3

We use tavilly for web searches, which is a free api tool for ai web scraping.

In [3]:
import os
import dotenv
dotenv.load_dotenv()

os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [4]:
### LLM

local_llm = 'llama3'

In [5]:
### Index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader
reader = DirectoryLoader("./docs/")
docs=reader.load()

# Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=HuggingFaceEmbeddings(),
)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/aditya/miniconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/aditya/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/aditya/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be 

In [6]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

In [8]:
### Generate

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise:
    Question: {question} 
    Context: {documents} 
    Answer: 
    """,
    input_variables=["question", "documents"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Chain
rag_chain = prompt | llm | StrOutputParser()



In [10]:
### Hallucination Grader 
# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation.
    
    Here are the facts:
    {documents} 

    Here is the answer: 
    {generation}

    output as a JSON with a single key 'score' and no preamble or explanation.
    """,
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()


In [12]:
### Answer Grader 

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     
    Here is the answer:
    {generation} 

    Here is the question: {question}
    """,
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()


In [14]:
### Router

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on the data labelled as "The DATA" below. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explaination. 
    
    Question to route: 
    {question}
    
    The DATA:
    {data}""",
    input_variables=["question","data"]
)

question_router = prompt | llm | JsonOutputParser()


In [16]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

We'll implement these as a control flow in LangGraph.

In [17]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents 
    """
    question : str
    generation : str
    web_search : str
    documents : List[str]

from langchain.schema import Document

### Nodes

def retrieve_and_route_question(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    state["documents"] = retriever.invoke(question)

    print(question)
    doc_txt = ""
    question = state["question"]
    for i in state["documents"]:  
        doc_txt+=i.page_content
    print(doc_txt)
    
    return {"documents":doc_txt,"question":question,}

def route_question(state):
    print("---ROUTE QUESTION---")
    source = question_router.invoke({"question": state["question"],"data":state["documents"]})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        state["documents"]=""
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    
    # RAG generation
    generation = rag_chain.invoke({"documents": documents, "question": question})
    print("in generate bro",documents,generation,sep="\n")
    return {"documents": documents, "question": question, "generation": generation}

def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    documents = web_results
    print("---WEB SEARCH--- DONE---")
    
    return {"documents": documents, "question": question}

### Conditional edge

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    print(documents)
    print("in some bro",documents,generation,sep="\n")

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    print("the json is ",score,sep=" ")
    print(documents,generation,sep="\n")
    grade = score['score']
    

    # Check hallucination
    if grade != "no":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve_and_route_question", retrieve_and_route_question) # retrieve_and_route_question
workflow.add_node("generate", generate) # generatae

### Graph Build

In [18]:
workflow.set_entry_point("retrieve_and_route_question")
workflow.add_conditional_edges(
    "retrieve_and_route_question",
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [19]:
# Compile
app = workflow.compile()


In [20]:


# Test
from pprint import pprint
inputs = {"question": "who is kendrick"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
who is kendrick
https://en.wikipedia.org/wiki/Kendrick_Lamar

Kendrick Lamar - Wikipedia

12. Ugwu, Reggie (February 3, 2015). "The

Radical Christianity Of Kendrick Lamar" (http s://web.archive.org/web/20160408011610/buz zfeed.com/reggieugwu/the-radical-christianity- of-kendrick-lamar). BuzzFeed. Archived from the original (https://www.buzzfeed.com/reggie ugwu/the-radical-christianity-of-kendrick-lama r) on April 8, 2016. Retrieved September 25, 2023.

13. Zisook, Brian (April 28, 2017). "Kendrickhttps://en.wikipedia.org/wiki/Kendrick_Lamar

Kendrick Lamar - Wikipediahttps://en.wikipedia.org/wiki/Kendrick_Lamar

Kendrick Lamar - Wikipediahttps://en.wikipedia.org/wiki/Kendrick_Lamar

Kendrick Lamar - Wikipedia
---ROUTE QUESTION---
{'datasource': 'vectorstore'}
vectorstore
---ROUTE QUESTION TO RAG---
'Finished running: retrieve_and_route_question:'
---GENERATE---
in generate bro
https://en.wikipedia.org/wiki/Kendrick_Lamar

Kendrick Lamar - Wikipedia

12. Ugwu, Reggie (Fe

Trace: 

https://smith.langchain.com/public/8d449b67-6bc4-4ecf-9153-759cd21df24f/r

In [21]:
inputs = {"question": "who is elon musk"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
who is elon musk
Saturday,[425] and is an angel investor of the music creation platform EngineEars.[426]332. Hiatt, Brian (August 9, 2017). "Kendrick Lamar: The Best Rapper Alive on Bono, Mandela, Stardom and More" (https://www.roll ingstone.com/music/music-features/kendrick-l amar-the-rolling-stone-interview-199817/). Rolling Stone. Retrieved October 20, 2023.448. Egkolfopoulou, Misyrlena (April 13, 2022). "Justin Bieber, Gwyneth Paltrow, Diplo and Drake have something in common: they all just invested in crypto-payments startup MoonPay" (https://fortune. com/crypto/2022/04/13/justin-bieber-gwyneth-paltrow-diplo-drake-investors-crypto-payments-start up-moonpay-nfts/). Fortune Crypto. Retrieved October 7, 2023.Despite becoming a prominent figure in popular culture, publications have noted Lamar's unconventional approach to celebrity culture.[399] He is notoriously reserved; he is reluctant to publicly discuss his personal life and generally avoids using social media.[400